In [110]:
import pandas as pd
import numpy as np
from collections import defaultdict

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

from sklearn.metrics import mean_squared_error

from surprise import Reader, Dataset, KNNBasic, SVD, accuracy
from surprise.model_selection import train_test_split as surprise_train_test_split

import warnings
warnings.filterwarnings('ignore')

In [111]:
ratings = pd.read_csv('data/FINAL-RATINGS.csv')
books = pd.read_csv('data/FINAL-BOOKS-WITH-TAGS.csv')

In [112]:
ratings.shape, books.shape

((1185991, 3), (10000, 9))

## Baseline model (popularity-based)

* **Purpose:** To establish a very simple benchmark. This model recommends the same most popular books to *every* user, regardless of their individual preferences.

* **Metrics Definition:** Explains the ranking metrics used throughout the notebook:

    * `Precision@K`: Out of the K books recommended, what fraction did the user actually rate highly (often defined as rating >= threshold like 4.0) in the hidden test set?

    * `Recall@K`: Out of all the books the user rated highly in the hidden test set, what fraction were captured within the top K recommendations?
    
    * `Hit Rate@K`: Did *at least one* of the user's highly-rated test set books appear in the top K recommendations? (Binary: 1 if yes, 0 if no, then averaged over users).

In [113]:
train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=42)

In [114]:
# Compute book popularity (rating counts)
popularity = (
    train_df['book_id']
    .value_counts()
    .reset_index(name='count')
    .rename(columns={'index':'book_id'})
)

In [115]:
# Top‑N popular books
N = 10

top_popular = (
    popularity
    .head(N)
    .merge(books[['book_id','title','average_rating']],
           on='book_id', how='left')
    [['book_id','title','average_rating','count']]
)

print(f"Top {N} Popular Books:")

top_popular.head(N)


Top 10 Popular Books:


,book_id,title,average_rating,count
0,1,"The Hunger Games (The Hunger Games, #1)",4.34,3632
1,2,Harry Potter and the Sorcerer's Stone (Harry P...,4.44,3487
2,4,To Kill a Mockingbird,4.25,3094
3,3,"Twilight (Twilight, #1)",3.57,2690
4,5,The Great Gatsby,3.89,2596
5,17,"Catching Fire (The Hunger Games, #2)",4.30,2587
6,20,"Mockingjay (The Hunger Games, #3)",4.03,2587
7,18,Harry Potter and the Prisoner of Azkaban (Harr...,4.53,2547
8,23,Harry Potter and the Chamber of Secrets (Harry...,4.37,2485
9,24,Harry Potter and the Goblet of Fire (Harry Pot...,4.53,2470


In [116]:
# RMSE
book_means = train_df.groupby('book_id')['rating'].mean()
global_mean = train_df['rating'].mean()
test_preds = test_df['book_id'].map(book_means).fillna(global_mean)
rmse_baseline = root_mean_squared_error(test_df['rating'], test_preds)

In [117]:
# Precision@N, Recall@N, HitRate@N

truth = test_df.groupby('user_id')['book_id'].apply(set).to_dict()

precisions, recalls, hits = [], [], []
for user, actual in truth.items():
    hit_count = len(actual & set(top_popular['book_id']))
    precisions.append(hit_count / N)
    recalls.append(hit_count / len(actual))
    hits.append(int(hit_count > 0))

In [118]:
print("\nBaseline Metrics:\n")

print(f"RMSE:         {np.mean(rmse_baseline):.4f}")
print(f"Precision@{N}: {np.mean(precisions):.4f}")
print(f"Recall@{N}:    {np.mean(recalls):.4f}")
print(f"HitRate@{N}:   {np.mean(hits):.4f}")



Baseline Metrics:

RMSE:         0.9562
Precision@10: 0.0669
Recall@10:    0.0302
HitRate@10:   0.4444


### new baseline using weighted rating

A more sophisticated baseline. It still recommends popular items globally, but it uses a weighted rating formula (like IMDb's) that balances a book's average rating with the number of ratings it has received. This prevents books with few high ratings from dominating purely popular books with many moderate ratings.

**IMDb Formula Implementation:**
* `m`: Calculates a minimum vote threshold (here, the 90th percentile of rating counts). Books with fewer ratings than `m` will have their scores adjusted more significantly towards the global average.
    
* `C`: The prior belief, set to the `global_mean` rating from the training set.

* `weighted_rating`: Applies the formula: `(v/(v+m))*R + (m/(v+m))*C`, where `v` is count, `R` is average rating.

**Top-N Weighted:** Selects the top N books based on this calculated `weighted_rating`.



In [119]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [120]:
# avg + count per book
pop_stats = (
    train_df
    .groupby('book_id')
    .agg(
        avg_rating=('rating', 'mean'),
        count_ratings=('rating', 'count')
    )
    .reset_index()
)

In [ ]:
# IMDb formula: (v/(v+m))*R + (m/(v+m))*C
m = pop_stats['count_ratings'].quantile(0.90)
C = train_df['rating'].mean()

pop_stats['weighted_rating'] = (
    (pop_stats['count_ratings'] / (pop_stats['count_ratings'] + m)) * pop_stats['avg_rating']
  + (m / (pop_stats['count_ratings'] + m)) * C
)

# top‑N by weighted rating
top_weighted_ids = pop_stats.nlargest(N, 'weighted_rating')['book_id'].tolist()



In [ ]:
preds_weighted = (
    test_df
    .merge(pop_stats[['book_id','weighted_rating']], on='book_id', how='left')
    ['weighted_rating']
    .fillna(C)          
)

rmse_weighted = rmse(test_df['rating'], preds_weighted)

In [123]:
# Precision, recall, hitrate calculation
truth = test_df.groupby('user_id')['book_id'].apply(set).to_dict()

prec_w, rec_w, hit_w = [], [], []

for actual in truth.values():
    hits = len(set(top_weighted_ids) & actual)
    prec_w.append(hits / N)
    rec_w.append(hits / len(actual) if actual else 0)
    hit_w.append(int(hits > 0))

precision_weighted = np.mean(prec_w)
recall_weighted    = np.mean(rec_w)
hitrate_weighted   = np.mean(hit_w)

In [124]:
# Print weighted‑rating baseline metrics
print("Weighted-rating baseline Metrics")
print(f"RMSE:          {rmse_weighted:.4f}")
print(f"Precision@{N}:  {precision_weighted:.4f}")
print(f"Recall@{N}:     {recall_weighted:.4f}")
print(f"HitRate@{N}:    {hitrate_weighted:.4f}")

Weighted-rating baseline Metrics
RMSE:          0.9635
Precision@10:  0.0437
Recall@10:     0.0195
HitRate@10:    0.3135


## Memory-based collaborative filtering using KNN

In [125]:
# Prepare Surprise dataset
reader = Reader(rating_scale = (1, 5))

data = Dataset.load_from_df(ratings[['user_id','book_id','rating']], reader)

trainset, testset = surprise_train_test_split(data, test_size = 0.2, random_state = 42)


In [ ]:
# Helper for top‑K metrics
def precision_recall_hit_at_k(preds, k = 10, thresh = 4.0):
    user_pred_true = defaultdict(list)
    for uid, iid, true, est, _ in preds:
        user_pred_true[uid].append((est, true))
    
    p_list, r_list, h_list = [], [], []

    for ratings in user_pred_true.values():
        ratings.sort(key = lambda x: x[0], reverse = True)
        top_k = ratings[:k]
        n_rel = sum(true >= thresh for _, true in ratings)
        n_rec = sum(true >= thresh for _, true in top_k)
        p_list.append(n_rec/k)
        r_list.append(n_rec/n_rel if n_rel else 0)
        h_list.append(int(n_rec > 0))
        
    return {
        'precision': np.mean(p_list),
        'recall':    np.mean(r_list),
        'hit_rate':  np.mean(h_list)
    }

results = []

### user-based collaborative filtering

Recommends items that *similar users* liked. Similarity is based on rating patterns.

In [127]:
algo_uu = KNNBasic(sim_options = {'name':'cosine','user_based': True}, 
                   k = 30, 
                   verbose = False)

algo_uu.fit(trainset)
pred_uu = algo_uu.test(testset)

rmse_uu = accuracy.rmse(pred_uu, verbose = False)
m_uu    = precision_recall_hit_at_k(pred_uu, k = 10, thresh = 4.0)

results.append({
    'Model':       'User-User CF',
    'RMSE':        rmse_uu,
    'Precision@10': m_uu['precision'],
    'Recall@10':    m_uu['recall'],
    'Hit@10':       m_uu['hit_rate']
})

### item-based collaborative filtering

Recommends items that are *similar* to items the user *already liked*. Similarity is based on users who rated both items similarly.

In [128]:
algo_ii = KNNBasic(sim_options = {'name':'cosine','user_based': False}, 
                   k = 30, 
                   verbose = False)

algo_ii.fit(trainset)
pred_ii = algo_ii.test(testset)

rmse_ii = accuracy.rmse(pred_ii, verbose = False)
m_ii    = precision_recall_hit_at_k(pred_ii, k = 10, thresh = 4.0)

results.append({
    'Model':       'Item-Item CF',
    'RMSE':        rmse_ii,
    'Precision@10': m_ii['precision'],
    'Recall@10':    m_ii['recall'],
    'Hit@10':       m_ii['hit_rate']
})


In [129]:
df_knn = pd.DataFrame(results)
print("\nKNN-based CF Comparison:\n")

df_knn.head()


KNN-based CF Comparison:



,Model,RMSE,Precision@10,Recall@10,Hit@10
0,User-User CF,0.958038,0.760124,0.531181,0.997335
1,Item-Item CF,0.892897,0.717934,0.501935,0.996763


## Model‑based CF using SVD matrix factorization

Uses Matrix Factorization - SVD -  to learn latent features (embeddings) for users and items. It predicts ratings by taking the dot product of user and item latent vectors.

### basic SVD

In [130]:
algo_svd = SVD(random_state = 42)
algo_svd.fit(trainset)
pred_svd = algo_svd.test(testset)

rmse_svd = accuracy.rmse(pred_svd, verbose = False)
metrics_svd = precision_recall_hit_at_k(pred_svd, k = 10, thresh = 4.0)
results.append({
    'Model':       'SVD (MF)',
    'RMSE':        rmse_svd,
    'Precision@10': metrics_svd['precision'],
    'Recall@10':    metrics_svd['recall'],
    'Hit@10':       metrics_svd['hit_rate']
})

### SVD with bias and parameters

- `n_factors = 50`: Number of latent dimensions to learn.

 - `biased = True`: Includes baseline estimates (user/item biases) in the prediction, often improving accuracy.

- `reg_all = 0.02`: Regularization term to prevent overfitting.
        
- `lr_all = 0.005`: Learning rate for the optimization algorithm (SGD).

In [134]:
algo_svd_bias = SVD(
    n_factors = 50,    # number of latent factors
    biased = True,     # include user/item biases
    reg_all = 0.02,    # regularization
    # lr = 0.005,    # learning rate
    random_state = 42
)

algo_svd_bias.fit(trainset)
pred_svd_bias = algo_svd_bias.test(testset)

rmse_svd = accuracy.rmse(pred_svd_bias, verbose = False)
metrics_svd = precision_recall_hit_at_k(pred_svd_bias, k = 10, thresh = 4.0)

results.append({
    'Model':       'SVD with bias (MF)',
    'RMSE':        rmse_svd,
    'Precision@10': metrics_svd['precision'],
    'Recall@10':    metrics_svd['recall'],
    'Hit@10':       metrics_svd['hit_rate']
})

In [135]:
df_svd = pd.DataFrame(results)

print("\nSVD based CF results:\n")

df_svd.head()


SVD based CF results:



,Model,RMSE,Precision@10,Recall@10,Hit@10
0,User-User CF,0.958038,0.760124,0.531181,0.997335
1,Item-Item CF,0.892897,0.717934,0.501935,0.996763
2,SVD (MF),0.849177,0.775478,0.544275,0.997715
3,SVD with bias (MF),0.847275,0.777363,0.545715,0.998001


### displaying recs for actual users

In [136]:
def get_top_n(predictions, n = 10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid in top_n:
        top_n[uid].sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = top_n[uid][:n]
    return top_n

top10_uu  = get_top_n(pred_uu,  n = 10)
top10_ii  = get_top_n(pred_ii,  n = 10)
top10_mf  = get_top_n(pred_svd, n = 10)
top10_mfb = get_top_n(pred_svd_bias, n = 10)

models = {
    'User-User CF': top10_uu,
    'Item-Item CF': top10_ii,
    'SVD (MF)'    : top10_mf,
    'SVD with bias (MF)': top10_mfb
}

# users
user_ids = [7531, 30102]

for uid in user_ids:
    key = uid if uid in top10_uu else str(uid)

    # coolect each model's top‑10 titles
    data = {}
    for name, top_dict in models.items():
        recs   = top_dict.get(key, [])
        titles = [
            books.loc[books.book_id == int(iid), 'title'].iloc[0]
            for iid, _ in recs
        ]
        titles += [''] * (10 - len(titles))
        data[name] = titles

    df = pd.DataFrame(data)
    print(f"\nUser {uid} Recommendations\n")
    display(df)



User 7531 Recommendations



,User-User CF,Item-Item CF,SVD (MF),SVD with bias (MF)
0,The Fault in Our Stars,Room,The Fault in Our Stars,"The Giver (The Giver, #1)"
1,Les Misérables,Snow Flower and the Secret Fan,"Catching Fire (The Hunger Games, #2)","Catching Fire (The Hunger Games, #2)"
2,Wonderstruck,Stones from the River,Honolulu,Room
3,Stones from the River,The Book of Ruth,Wonderstruck,Wonderstruck
4,Let's Pretend This Never Happened: A Mostly Tr...,Wonderstruck,"The Giver (The Giver, #1)",Moloka'i
5,"The English Spy (Gabriel Allon, #15)","Catching Fire (The Hunger Games, #2)",The Light Between Oceans,Les Misérables
6,Room,The Fault in Our Stars,Moloka'i,The Fault in Our Stars
7,Moloka'i,The Girl on the Train,"The English Spy (Gabriel Allon, #15)","The English Spy (Gabriel Allon, #15)"
8,Honolulu,Les Misérables,Les Misérables,Snow Flower and the Secret Fan
9,Shanghai Girls (Shanghai Girls #1),Honolulu,Let's Pretend This Never Happened: A Mostly Tr...,The Girl on the Train



User 30102 Recommendations



,User-User CF,Item-Item CF,SVD (MF),SVD with bias (MF)
0,"Fruits Basket, Vol. 4",Hedda Gabler,Watchmen,Watchmen
1,"Fruits Basket, Vol. 3","Hana-Kimi, Vol. 1 (Hana-Kimi, #1)","Fruits Basket, Vol. 4",The Call of the Wild
2,"One Grave at a Time (Night Huntress, #6)",Watchmen,The Call of Cthulhu and Other Weird Stories,Animal Farm
3,The Call of Cthulhu and Other Weird Stories,Romeo and Juliet,A Streetcar Named Desire,"Fruits Basket, Vol. 3"
4,Watchmen,The Call of the Wild,Animal Farm,Romeo and Juliet
5,Animal Farm,The Great Gatsby,"Fruits Basket, Vol. 3",A Streetcar Named Desire
6,"Hana-Kimi, Vol. 1 (Hana-Kimi, #1)","Halfway to the Grave (Night Huntress, #1)",Romeo and Juliet,"Halfway to the Grave (Night Huntress, #1)"
7,Eternal Kiss of Darkness (Night Huntress World...,"Fruits Basket, Vol. 4","Bloody Bones (Anita Blake, Vampire Hunter #5)",The Call of Cthulhu and Other Weird Stories
8,"The Lunatic Cafe (Anita Blake, Vampire Hunter #4)","Fruits Basket, Vol. 3",The Indian in the Cupboard (The Indian in the ...,"One Grave at a Time (Night Huntress, #6)"
9,A Streetcar Named Desire,The Call of Cthulhu and Other Weird Stories,"One Grave at a Time (Night Huntress, #6)","Fruits Basket, Vol. 4"


## content-based filtering

**Purpose:** recommends items based on their textual content (description, tags) similarity to items a user liked previously. It doesn't rely on other users' behavior.

In [137]:
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel

import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anusha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [138]:
# concat description + tags to create a new field
books['content'] = (
    books['description'].fillna('') + ' ' +
    books['unique_tags_str'].fillna('')
)


def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)     # strip punctuation
    tokens = text.split()
    tokens = [tok for tok in tokens if tok not in stop_words]
    return ' '.join(tokens)

books['content_clean'] = books['content'].apply(clean_text)


In [139]:
# compute tf‑idf & cosine similarity

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(books['content_clean'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(f"Cosine similarity matrix shape: {cosine_sim.shape}")


Cosine similarity matrix shape: (10000, 10000)


`cosine_sim` is a matrix where `cosine_sim[i][j]` is the content similarity between book `i` and book `j`.

In [140]:
# mapping from book_id to index 
book_to_idx = {bid: idx for idx, bid in enumerate(books['book_id'])}
idx_to_book = {idx: bid for bid, idx in book_to_idx.items()}

In [141]:
# function to get liked titles & recommendations
def content_recommendations_for_user(user_id, ratings_df, books_df,
                                     cosine_sim, book_to_idx,
                                     top_n = 10, threshold = 4.0):
    
    # find books the user rated ≥ threshold
    liked_ids = ratings_df[
        (ratings_df.user_id == user_id) &
        (ratings_df.rating  >= threshold)
    ]['book_id'].unique()
    
    liked_titles = books_df.loc[
        books_df.book_id.isin(liked_ids),
        ['book_id','title']
    ]
    
    # aggregate similarity scores
    agg_scores = {}
    for bid in liked_ids:
        idx = book_to_idx.get(bid)
        if idx is None: 
            continue
        for other_idx, score in enumerate(cosine_sim[idx]):
            agg_scores[other_idx] = agg_scores.get(other_idx, 0) + score
    
    # remove already‑seen books
    for bid in liked_ids:
        idx = book_to_idx.get(bid)
        if idx is not None:
            agg_scores.pop(idx, None)
    
    # select top‑N
    top_indices = sorted(
        agg_scores.keys(),
        key = lambda i: agg_scores[i],
        reverse = True
    )[:top_n]

    rec_book_ids = [idx_to_book[idx] for idx in top_indices]
    rec_titles = books_df.loc[
        books_df.book_id.isin(rec_book_ids),
        ['book_id','title']
    ]
    
    return liked_titles, rec_titles

In [ ]:
# Example for user 3981
user_id = 3981
liked, recs = content_recommendations_for_user(
    user_id, ratings, books,
    cosine_sim, book_to_idx,
    top_n = 10, threshold = 4.0
)

print(f"User {user_id} liked these books:")
display(liked)

print(f"\nTop-10 content-based recommendations for user {user_id}:")
display(recs)

User 3981 liked these books:


,book_id,title
38,39,"A Game of Thrones (A Song of Ice and Fire, #1)"
47,48,Fahrenheit 451
64,65,Slaughterhouse-Five
83,84,"Jurassic Park (Jurassic Park, #1)"
103,104,The Road
...,...,...
8879,8880,"Transmetropolitan, Vol. 5: Lonely City (Transm..."
8960,8961,Footfall
9427,9428,Johnny Mnemonic
9606,9607,"Singularity Sky (Eschaton, #1)"



Top-10 content-based recommendations for user 3981:


,book_id,title
509,510,"The Great Hunt (Wheel of Time, #2)"
971,972,Journey to the Center of the Earth (Extraordin...
1118,1119,"A Crown of Swords (Wheel of Time, #7)"
1203,1204,The Invisible Man
1342,1343,"The Light Fantastic (Discworld, #2; Rincewind #2)"
4584,4585,Foundation and Chaos (Second Foundation Trilog...
6014,6015,"The Atlantis World (The Origin Mystery, #3)"
6430,6431,"Path of Destruction (Star Wars: Darth Bane, #1)"
7920,7921,2BR02B
9407,9408,"The Darkest Road (The Fionavar Tapestry, #3)"


In [142]:
# Example for user 7531
user_id = 7531
liked, recs = content_recommendations_for_user(
    user_id, ratings, books,
    cosine_sim, book_to_idx,
    top_n = 10, threshold = 4.0
)

print(f"User {user_id} liked these books:")
display(liked)

print(f"\nTop-10 content-based recommendations for user {user_id}:")
display(recs)

User 7531 liked these books:


,book_id,title
0,1,"The Hunger Games (The Hunger Games, #1)"
5,6,The Fault in Our Stars
7,8,The Catcher in the Rye
10,11,The Kite Runner
15,16,"The Girl with the Dragon Tattoo (Millennium, #1)"
...,...,...
6046,6047,Honolulu
6280,6281,"The English Spy (Gabriel Allon, #15)"
7043,7044,The Ship of Brides
7774,7775,"Love and War (North and South, #2)"



Top-10 content-based recommendations for user 7531:


,book_id,title
971,972,Journey to the Center of the Earth (Extraordin...
1738,1739,The Bonfire of the Vanities
3437,3438,فلتغفري
4263,4264,ذاكرة الجسد
4299,4300,The Ladies' Room
5930,5931,Adam Bede
7042,7043,نادي السيارات
7837,7838,Lady Windermere's Fan
9771,9772,The Amateur Marriage
9899,9900,Guts


In [143]:
# 30102
user_id = 30102
liked, recs = content_recommendations_for_user(
    user_id, ratings, books,
    cosine_sim, book_to_idx,
    top_n = 10, threshold = 4.0
)

print(f"User {user_id} liked these books:")
display(liked)

print(f"\nTop-10 content-based recommendations for user {user_id}:")
display(recs)

User 30102 liked these books:


,book_id,title
1,2,Harry Potter and the Sorcerer's Stone (Harry P...
3,4,To Kill a Mockingbird
4,5,The Great Gatsby
12,13,1984
13,14,Animal Farm
...,...,...
9318,9319,"Ouran High School Host Club, Vol. 15 (Ouran Hi..."
9456,9457,"Fruits Basket, Vol. 14"
9535,9536,"Fruits Basket, Vol. 15"
9672,9673,"Fruits Basket, Vol. 3"



Top-10 content-based recommendations for user 30102:


,book_id,title
971,972,Journey to the Center of the Earth (Extraordin...
3265,3266,NARUTO -ナルト- 巻ノ四十三
4414,4415,美少女戦士セーラームーン新装版 1 [Bishōjo Senshi Sailor Moon ...
4929,4930,Pandora Hearts 1巻
6026,6027,"Naruto, Vol. 11: Impassioned Efforts (Naruto, ..."
6140,6141,Harry Potter and the Order of the Phoenix (Har...
6618,6619,"The Face Of Deception (Eve Duncan, #1)"
8213,8214,"Sin City, Vol. 3: The Big Fat Kill (Sin City, #3)"
9769,9770,مخطوطة بن إسحاق: مدينة الموتى
9899,9900,Guts


### content-based evaluation using leave-one-out

to evaluate the content-based approach using RMSE and ranking metrics,leave-one-out splitting is used: for each user, one rating is held out for testing, and the rest are used for training/profile building.


In [144]:
# mapping from book_id to index
book_to_idx = pd.Series(books.index, index = books['book_id']).to_dict()
idx_to_book = {idx: bid for bid, idx in book_to_idx.items()}

def leave_one_out(df, user_col = 'user_id', item_col = 'book_id'):

    train_parts, test_parts = [], []

    for uid, grp in df.groupby(user_col):
        if len(grp) < 2:
            train_parts.append(grp)
        else:
            grp_shuffled = grp.sample(frac = 1, random_state = 42)
            test_parts.append(grp_shuffled.iloc[:1])
            train_parts.append(grp_shuffled.iloc[1:])

    train_df = pd.concat(train_parts).reset_index(drop = True)
    test_df  = pd.concat(test_parts).reset_index(drop = True)
    
    return train_df, test_df

train_df, test_df = leave_one_out(ratings)

In [145]:
# build user‑ratings dict & global mean
user_ratings = train_df.groupby('user_id').apply(
    lambda g: dict(zip(g['book_id'], g['rating']))
).to_dict()

GLOBAL_MEAN = train_df['rating'].mean()


In [146]:
# Rating predictor via Content Similarity
def predict_rating(user_id, book_id, k=10):
    if user_id not in user_ratings or book_id not in book_to_idx:
        return GLOBAL_MEAN

    idx = book_to_idx[book_id]
    sims = list(enumerate(cosine_sim[idx]))
    sims.sort(key=lambda x: x[1], reverse=True)

    num, den, count = 0.0, 0.0, 0
    for other_idx, score in sims:
        other_bid = idx_to_book[other_idx]
        if other_bid in user_ratings[user_id]:
            r = user_ratings[user_id][other_bid]
            num += score * r
            den += abs(score)
            count += 1
            if count >= k:
                break

    return (num / den) if den > 0 else GLOBAL_MEAN



In [147]:
# Evaluate RMSE on the test set
preds = test_df.apply(lambda row: predict_rating(row['user_id'], row['book_id']), axis=1)
rmse_value = rmse((test_df['rating'], preds))
print(f"Content-based RMSE: {rmse_value:.4f}")

TypeError: rmse() missing 1 required positional argument: 'y_pred'

In [ ]:
# Top‑K Recommendations for Ranking Metrics
def top_k_recs_for_user(user_id, K = 10):
    if user_id not in user_ratings:
        return []
    agg_scores = {}
    for bid, _ in user_ratings[user_id].items():
        if bid not in book_to_idx:
            continue
        idx = book_to_idx[bid]
        for j, score in enumerate(cosine_sim[idx]):
            agg_scores[j] = agg_scores.get(j, 0) + score

    seen = set(user_ratings[user_id])
    candidates = [(j, s) for j, s in agg_scores.items() if idx_to_book[j] not in seen]
    top = sorted(candidates, key=lambda x: x[1], reverse=True)[:K]
    return [idx_to_book[j] for j, _ in top]

# Prepare ground‑truth relevant items (rating ≥ 4.0)
relevants = test_df[test_df['rating'] >= 4.0].groupby('user_id')['book_id'].apply(set).to_dict()



In [ ]:
# Ranking Evaluation Metrics
def evaluate_ranking(K = 10):
    precisions, recalls, hits, APs = [], [], [], []

    for uid, true_set in relevants.items():
        recs = top_k_recs_for_user(uid, K)
        if not true_set:
            continue

        # Hit rate
        hit = int(bool(set(recs) & true_set))
        hits.append(hit)

        # Precision & Recall
        match_count = sum(1 for b in recs if b in true_set)
        precisions.append(match_count / K)
        recalls.append(match_count / len(true_set))

        # Average Precision
        num_hits, sum_prec = 0, 0.0
        for i, b in enumerate(recs, start=1):
            if b in true_set:
                num_hits += 1
                sum_prec += num_hits / i
        APs.append(sum_prec / min(len(true_set), K))

    return {
        'HitRate@K': np.mean(hits),
        'Precision@K': np.mean(precisions),
        'Recall@K': np.mean(recalls),
        'MAP@K': np.mean(APs)
    }

In [ ]:
metrics = evaluate_ranking(K = 10)
print("\nRanking Metrics:")
for name, val in metrics.items():
    print(f"{name}: {val:.4f}")

In [148]:
# function to get top‑10 similar books by content

def get_similar_books(book_id, K = 10):
    if book_id not in book_to_idx:
        print(f"Book ID {book_id} not found in index.")
        return pd.DataFrame(columns=['book_id','title'])
    
    idx = book_to_idx[book_id]
    
    sim_scores = list(enumerate(cosine_sim[idx]))   
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)[1:K+1]
    rec_ids = [idx_to_book[j] for j, _ in sim_scores]
    
    return books.loc[
        books['book_id'].isin(rec_ids),
        ['book_id','title']
    ].reset_index(drop = True)

In [163]:
target_id = 3398   # replace with any book_id 

target_title = books.loc[books['book_id'] == target_id, 'title'].iloc[0]

recommendations = get_similar_books(target_id, K = 10)

print(f"Target book (ID: {target_id}): {target_title}\n")
print(f"Top 10 books similar to '{target_title}':")
recommendations.head(10)


Target book (ID: 3398): The Ugly Duckling

Top 10 books similar to 'The Ugly Duckling':


,book_id,title
0,1120,Aesop's Fables
1,4291,Alice's Adventures in Wonderland: A Pop-Up Ada...
2,5718,Beauty and the Beast
3,6514,The Hat
4,6526,Kitten's First Full Moon
5,6778,Madeline and the Bad Hat
6,7298,Rikki-Tikki-Tavi
7,7481,"When We Were Very Young (Winnie-the-Pooh, #3)"
8,8192,The Giraffe and the Pelly and Me
9,8665,The Country Mouse and the City Mouse; The Fox ...


##  Two‑Tower Recommendation System

In [178]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence


In [174]:
device = torch.device('mps' if torch.mps.is_available() else 'cpu')

### preprocess: build user history and ID mappings

In [175]:
# mappings
user2idx = {u: i for i, u in enumerate(ratings['user_id'].unique())}
book2idx = {b: i for i, b in enumerate(books['book_id'].unique())}

# user history: list of past rated book indices for each user
user_hist = ratings.groupby('user_id')['book_id'].apply(list).to_dict()

# convert book_ids in history to indices
user_hist_idx = {user2idx[u]: [book2idx[b] for b in hist] for u, hist in user_hist.items()}



### dataset & dataLlader

In [176]:
class TwoTowerDataset(Dataset):
    def __init__(self, ratings_df):
        self.data = ratings_df

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        uid = user2idx[row['user_id']]
        bid = book2idx[row['book_id']]
        hist = user_hist_idx.get(uid, [])
        rating = torch.tensor(row['rating'], dtype = torch.float)
        return uid, torch.tensor(hist, dtype = torch.long), bid, rating

In [177]:
train_df, test_df = train_test_split(ratings, test_size = 0.2, random_state = 42)
train_ds = TwoTowerDataset(train_df)

test_ds  = TwoTowerDataset(test_df)
train_loader = DataLoader(train_ds, batch_size = 256, shuffle = True, collate_fn = lambda batch: batch)

### model architecture


In [179]:
class TwoTowerModel(nn.Module):

    def __init__(self, num_users, num_books, emb_dim = 64, hist_dim = 64):
        super().__init__()

        # embeddings
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.book_emb = nn.Embedding(num_books, emb_dim)
        
        # Towers
        # User tower: combine user ID + mean history
        self.user_mlp = nn.Sequential(
            nn.Linear(emb_dim * 2, hist_dim),
            nn.ReLU(),
            nn.Linear(hist_dim, emb_dim)
        )

        # Item tower: book embedding
        self.item_mlp = nn.Sequential(
            nn.Linear(emb_dim, hist_dim),
            nn.ReLU(),
            nn.Linear(hist_dim, emb_dim)
        )

    def forward(self, user_ids, histories, book_ids):
        # user ID embedding
        u = self.user_emb(user_ids)  

        # 1) turn each history list into a tensor on the right device
        hist_tensors = [
            torch.tensor(h, dtype = torch.long, device = device)
            for h in histories
        ]

        # 2) pad them (batch_first) on the same device
        padded = pad_sequence(hist_tensors, batch_first=True, padding_value = 0)  

        # lookup embeddings and take mean
        h_emb = self.book_emb(padded)     
        h_mean = h_emb.mean(dim = 1)        


  

        # combine
        u_concat = torch.cat([u, h_mean], dim=1)  
        u_vec = self.user_mlp(u_concat)          

        # item tower
        i = self.book_emb(book_ids)
        i_vec = self.item_mlp(i)  

        score = (u_vec * i_vec).sum(dim=1)
        return score



In [180]:
# instantiate model
num_users = len(user2idx)
num_books = len(book2idx)
model = TwoTowerModel(num_users, num_books)

### training loop


In [181]:
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

for epoch in range(5):
    model.train()
    total_loss = 0
    for batch in train_loader:
        user_ids, histories, book_ids, ratings_true = zip(*batch)
        user_ids = torch.tensor(user_ids, device=device)
        book_ids = torch.tensor(book_ids, device=device)
        ratings_true = torch.stack(ratings_true).to(device)

        optimizer.zero_grad()
        preds = model(user_ids, histories, book_ids)
        loss = criterion(preds, ratings_true)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1} — Train MSE: {total_loss/len(train_loader):.4f}")

Epoch 1 — Train MSE: 0.9174
Epoch 2 — Train MSE: 0.7642
Epoch 3 — Train MSE: 0.7451
Epoch 4 — Train MSE: 0.7362
Epoch 5 — Train MSE: 0.7285


### inference

In [184]:
# Precompute item embeddings
model.eval()
with torch.no_grad():
    all_book_ids = torch.arange(num_books, device = device)
    item_embs = model.item_mlp(model.book_emb(all_book_ids)) 

# top-10 for a given user
def recommend_user(user_id, K = 10):
    uid = user2idx[user_id]

    # get user vector
    u_vec = model.user_mlp(torch.cat([
        model.user_emb(torch.tensor([uid], device=device)),
        model.book_emb(torch.tensor(user_hist_idx[uid], device=device)).mean(dim=0, keepdim=True)
    ], dim=1))  

    # compute scores
    scores = torch.matmul(item_embs, u_vec.squeeze())
    topk = torch.topk(scores, K).indices.cpu().numpy()
    return books.iloc[topk][['book_id','title']]

In [185]:
print(recommend_user(ratings['user_id'].iloc[0], K = 10))

      book_id                                              title
5206     5207  The Days Are Just Packed: A Calvin and Hobbes ...
8945     8946                                          The Divan
9075     9076    Preach My Gospel: A Guide To Missionary Service
1787     1788       The Calvin and Hobbes Tenth Anniversary Book
8547     8548                                 This is Not My Hat
8662     8663                 Locke & Key, Vol. 6: Alpha & Omega
4482     4483  It's a Magical World: A Calvin and Hobbes Coll...
3627     3628                     The Complete Calvin and Hobbes
779       780                                  Calvin and Hobbes
6750     6751  The Declaration of Independence and The Consti...


**Overall Goal:**

The primary goal is to compare different recommendation algorithms based on their ability to predict user ratings (accuracy) and suggest relevant books (ranking quality). It starts with simple baselines and progresses to more complex collaborative filtering, content-based, and deep learning methods.

**1. Initial Setup & Data Loading**

* **Load Data:**
    * `ratings = pd.read_csv('data/FINAL-RATINGS.csv')`: Loads user ratings data (likely containing `user_id`, `book_id`, `rating`).
    * `books = pd.read_csv('data/FINAL-BOOKS-WITH-TAGS.csv')`: Loads book metadata (likely containing `book_id`, `title`, `description`, `tags`, etc.).
    * `ratings.shape, books.shape`: Prints the dimensions (rows, columns) of the loaded DataFrames to get an idea of the data size.

**2. Baseline Model: Popularity-Based**

* **Purpose:** To establish a very simple benchmark. This model recommends the same most popular books to *every* user, regardless of their individual preferences.
* **Metrics Definition:** Explains the ranking metrics used throughout the notebook:
    * `Precision@K`: Out of the K books recommended, what fraction did the user actually rate highly (often defined as rating >= threshold like 4.0) in the hidden test set?
    * `Recall@K`: Out of all the books the user rated highly in the hidden test set, what fraction were captured within the top K recommendations?
    * `Hit Rate@K`: Did *at least one* of the user's highly-rated test set books appear in the top K recommendations? (Binary: 1 if yes, 0 if no, then averaged over users).
* **Train/Test Split (Standard):** `train_test_split(ratings, ...)` splits the `ratings` data (80% train, 20% test) so the model's performance can be evaluated on unseen data. `random_state=42` ensures reproducibility.
* **Calculate Popularity:**
    * `train_df['book_id'].value_counts()`: Counts how many times each `book_id` appears in the *training* data. This count represents popularity.
    * The result is stored in the `popularity` DataFrame.
* **Get Top N Popular:**
    * `N = 10`: Sets the number of recommendations to generate (K=10).
    * `popularity.head(N)`: Selects the N most frequent (popular) books.
    * `.merge(...)`: Joins this with the `books` DataFrame to get the titles and average ratings for these popular books.
    * `print(...)`, `top_popular.head(N)`: Displays the top N popular books.
* **Evaluate Popularity Baseline:**
    * **RMSE:**
        * `book_means`: Calculates the average rating for each book based *only* on the training data.
        * `global_mean`: Calculates the overall average rating across *all* ratings in the training data.
        * `test_preds`: For each rating in the test set, it predicts the rating as the book's average rating (from `book_means`). If the book wasn't in the training set (no average rating available), it predicts the `global_mean`.
        * `rmse_baseline`: Calculates RMSE between the predicted ratings and the actual ratings in the test set.
    * **Precision@N, Recall@N, HitRate@N:**
        * `truth`: Creates a dictionary where keys are `user_id`s and values are *sets* of `book_id`s that the user rated in the *test* set. This is the "ground truth" for ranking evaluation.
        * Loop: Iterates through each user and their actual test set books (`actual`).
        * `hit_count`: Calculates how many books from the `top_popular` list are *also* present in the user's `actual` test set books.
        * Metrics Calculation: Calculates Precision (`hit_count / N`), Recall (`hit_count / len(actual)`), and Hit Rate (`int(hit_count > 0)`) for *each user*.
        * `np.mean(...)`: Averages these metrics across all users in the test set.
    * `print(...)`: Displays the calculated baseline metrics.

**3. Baseline Model: Weighted Rating (IMDb Style)**

* **Purpose:** A slightly more sophisticated baseline. It still recommends popular items globally, but it uses a weighted rating formula (like IMDb's) that balances a book's average rating with the number of ratings it has received. This prevents books with few high ratings from dominating purely popular books with many moderate ratings.
* **RMSE Function:** Defines a helper function `rmse`.
* **Calculate Stats:** `pop_stats` DataFrame calculates the average rating (`avg_rating`) and rating count (`count_ratings`) for each book from the training data.
* **IMDb Formula Implementation:**
    * `m`: Calculates a minimum vote threshold (here, the 90th percentile of rating counts). Books with fewer ratings than `m` will have their scores adjusted more significantly towards the global average.
    * `C`: The prior belief, set to the `global_mean` rating from the training set.
    * `weighted_rating`: Applies the formula: `(v/(v+m))*R + (m/(v+m))*C`, where `v` is count, `R` is average rating.
* **Top-N Weighted:** Selects the top N books based on this calculated `weighted_rating`.
* **Evaluation (RMSE, Precision, Recall, HitRate):**
    * Calculates RMSE using the `weighted_rating` as the prediction (falling back to `C` if a book isn't found).
    * Calculates Precision, Recall, and HitRate similarly to the first baseline, but compares against the `top_weighted_ids` list.
* **Print Results:** Displays the weighted-rating baseline metrics.
* **Comparative Metrics:** Prints a formatted table comparing the simple popularity and weighted rating baselines side-by-side.

**4. Memory-Based Collaborative Filtering (CF) using KNN**

* **Purpose:** Moves beyond global recommendations to personalized ones based on user/item similarity.
    * **User-User CF:** Recommends items that *similar users* liked. Similarity is based on rating patterns.
    * **Item-Item CF:** Recommends items that are *similar* to items the user *already liked*. Similarity is based on users who rated both items similarly.
* **Prepare Surprise Data:**
    * `Reader`: Defines the rating scale (1 to 5).
    * `Dataset.load_from_df`: Loads the `ratings` DataFrame (selecting only relevant columns) into Surprise's internal data format.
    * `surprise_train_test_split`: Splits the Surprise `data` object into `trainset` and `testset`.
* **`precision_recall_hit_at_k` Helper Function:**
    * This function is crucial for evaluating ranking metrics with Surprise models.
    * It takes Surprise's prediction output (`preds`, which is a list of tuples: `uid, iid, true_rating, estimated_rating, details`).
    * Groups predictions by user (`user_pred_true`).
    * For each user:
        * Sorts their predicted items by the `estimated_rating` in descending order.
        * Selects the `top_k` items.
        * Calculates `n_rel`: Total number of *actually relevant* items for this user in the test set (true rating >= threshold).
        * Calculates `n_rec`: Number of *recommended relevant* items in the `top_k`.
        * Calculates and stores Precision@K, Recall@K, and HitRate@K for that user.
    * Returns the average metrics across all users.
* **Results List:** `results = []` initializes a list to store evaluation metrics for different models.
* **User-User CF (`algo_uu`):**
    * `KNNBasic`: Instantiates the K-Nearest Neighbors algorithm from Surprise.
    * `sim_options`: Specifies similarity calculation (cosine similarity, `user_based=True`).
    * `k=30`: Use 30 nearest neighbors.
    * `verbose=False`: Suppresses training output.
    * `algo_uu.fit(trainset)`: Trains the model on the training data.
    * `pred_uu = algo_uu.test(testset)`: Makes predictions on the test set.
    * `rmse_uu = accuracy.rmse(...)`: Calculates RMSE using Surprise's accuracy module.
    * `m_uu = precision_recall_hit_at_k(...)`: Calculates ranking metrics using the helper function.
    * Appends the results (model name, RMSE, P@10, R@10, H@10) to the `results` list.
* **Item-Item CF (`algo_ii`):**
    * Same process as User-User, but with `user_based=False` in `sim_options`.
* **Display KNN Results:** Creates a Pandas DataFrame `df_knn` from the `results` list and displays the comparison.

**5. Model-Based Collaborative Filtering using SVD**

* **Purpose:** Uses Matrix Factorization (specifically Singular Value Decomposition - SVD) to learn latent features (embeddings) for users and items. It predicts ratings by taking the dot product of user and item latent vectors. Often performs better than KNN, especially on sparse data.
* **Basic SVD (`algo_svd`):**
    * Instantiates a basic `SVD` model from Surprise.
    * Fits, tests, calculates RMSE and ranking metrics, and appends to `results`.
* **SVD with Bias & Parameters (`algo_svd` - overwritten):**
    * Instantiates `SVD` again, but with common tuning parameters:
        * `n_factors=50`: Number of latent dimensions to learn.
        * `biased=True`: Includes baseline estimates (user/item biases) in the prediction, often improving accuracy.
        * `reg_all=0.02`: Regularization term to prevent overfitting.
        * `lr_all=0.005`: Learning rate for the optimization algorithm (stochastic gradient descent).
    * Fits, tests, calculates metrics (`pred_svd_bias`), and appends results. *Note: This overwrites the previous `algo_svd` variable, but the results from the basic SVD were already stored.*
* **Display SVD Results:** Creates `df_svd` (using the updated `results` list containing UU, II, SVD, SVD-Bias results) and displays it.

**6. Displaying Recommendations**

* **Purpose:** To show concrete examples of the recommendations generated by the different trained models for specific users.
* **`get_top_n` Function:**
    * Takes the prediction output from a Surprise model.
    * Groups predictions by user.
    * For each user, sorts their predicted items by estimated rating.
    * Returns a dictionary mapping `user_id` to a list of their top `n` recommended (`book_id`, `estimated_rating`) tuples.
* **Generate Top-10 Lists:** Calls `get_top_n` for each of the four CF models trained (User-User, Item-Item, SVD, SVD-Bias).
* **Prepare for Display:**
    * `models`: A dictionary to easily access the top-10 lists for each model.
    * `user_ids`: A list of specific users to generate recommendations for.
* **Loop and Display:**
    * Iterates through the selected `user_ids`.
    * For each user, it iterates through the `models`.
    * Retrieves the top-10 recommended `book_id`s for the current model and user.
    * Looks up the corresponding book `title`s from the `books` DataFrame.
    * Stores these titles in a dictionary `data`.
    * Creates a DataFrame `df_side` where columns are model names and rows are the recommended titles (rank 1 to 10).
    * Uses `display(df_side)` (specific to Jupyter environments) to show the recommendations side-by-side for that user.

**7. Markdown Summary Cells**

* These cells (`## Model Comparison`, `## 1. RMSE`, etc.) use Markdown formatting to:
    * Present the final evaluation metrics (RMSE, Precision@10, Recall@10, Hit@10) from the `df_svd` DataFrame in clear tables.
    * Reiterate the interpretation of RMSE (lower is better).
    * Show Top-5 metrics (requires recalculating or adjusting the `precision_recall_hit_at_k` function for K=5, which isn't explicitly shown here but the results are presented).
    * Explain *why* SVD often performs well (latent factors, regularization, handling sparsity).
    * Show a sample Top-5 recommendation list for one user (taken from one of the models, likely User-User based on the heading).
    * Provide a final summary comparing models and recommending SVD for accuracy/ranking and Item-Item for interpretability.

**8. Content-Based Filtering**

* **Purpose:** Recommends items based on their textual content (description, tags) similarity to items a user liked previously. It doesn't rely on other users' behavior.
* **Content Field:** Creates a combined 'content' column from book descriptions and tags.
* **Text Cleaning:**
    * Downloads `stopwords` from `nltk`.
    * Defines `clean_text` function: converts to lowercase, removes punctuation/non-alphanumeric characters (except spaces), splits into tokens, removes common English stopwords.
    * Applies cleaning to the 'content' column -> 'content_clean'.
* **TF-IDF Vectorization:**
    * `TfidfVectorizer`: Initializes the vectorizer. TF-IDF (Term Frequency-Inverse Document Frequency) converts text into numerical vectors, giving higher weight to words that are frequent in a specific document but rare across all documents.
    * `tfidf.fit_transform()`: Learns the vocabulary and IDF weights from 'content_clean' and transforms the text into a sparse TF-IDF matrix (`tfidf_matrix`).
* **Cosine Similarity:**
    * `linear_kernel(tfidf_matrix, tfidf_matrix)`: Efficiently computes the cosine similarity between all pairs of book TF-IDF vectors. The result `cosine_sim` is a matrix where `cosine_sim[i][j]` is the content similarity between book `i` and book `j`.
    * `book_idx`, `book_to_idx`, `idx_to_book`: Creates mappings between `book_id` and the row index in the `books` DataFrame / `cosine_sim` matrix.
* **Generating Recommendations (Example User):**
    * Picks a `sample_user`.
    * Finds `liked_book_ids` (rated >= 4.0) for that user from the *full* `ratings` dataset.
    * `agg_scores`: Calculates aggregate similarity scores. It iterates through the user's liked books. For each liked book, it adds its similarity scores (from `cosine_sim`) to all *other* books into the `agg_scores` dictionary. Books similar to multiple liked books get higher scores.
    * Removes books the user has already seen (`liked_book_ids`) from `agg_scores`.
    * Sorts the remaining books by aggregated score and gets the top `N` indices.
    * Looks up the titles and displays the recommendations.
* **Content-Based Evaluation (Leave-One-Out):**
    * **Purpose:** To evaluate the content-based approach using RMSE and ranking metrics, similar to the CF models. Leave-One-Out (LOO) splitting is used: for each user, one rating is held out for testing, and the rest are used for training/profile building.
    * **LOO Split Function:** Defines `leave_one_out` to perform this split.
    * Applies LOO to the `ratings` data.
    * `user_ratings`: Creates a dictionary mapping users to their *training* ratings.
    * `GLOBAL_MEAN`: Calculated from the *training* data.
    * **`predict_rating` Function:**
        * Predicts the rating for a *test* user-item pair.
        * Finds the `k` most similar items (based on `cosine_sim`) to the target item *that the user has rated in their training history*.
        * Predicts the rating as a weighted average of the ratings given to those similar items, where the weight is the cosine similarity score. Falls back to `GLOBAL_MEAN` if needed.
    * **RMSE Evaluation:** Applies `predict_rating` to the LOO test set and calculates RMSE.
    * **`top_k_recs_for_user` Function (Ranking):** Generates top-K recommendations for a user based on aggregating content similarity scores from their *training* history (similar to the example).
    * `relevants`: Defines the ground truth for ranking – items in the *test* set rated >= 4.0.
    * **`evaluate_ranking` Function:** Calculates HitRate@K, Precision@K, Recall@K, and MAP@K (Mean Average Precision - considers the rank of correct items) using the LOO test setup.
    * Calls `evaluate_ranking` and prints the results.
* **`get_similar_books` Function (Item-to-Item Content Similarity):**
    * A utility function to find the K most similar books to a *given* book based purely on content (`cosine_sim`).
    * Used for demonstrating item-based content similarity.

**9. Two-Tower Recommendation System (Deep Learning)**

* **Purpose:** Implements a more advanced deep learning model. It learns separate representations (embeddings) for users and items in two parallel "towers" (neural networks) and predicts compatibility using their outputs. This is common for large-scale industrial systems.
* **Imports:** `torch` and related modules for deep learning.
* **Preprocessing:**
    * `user2idx`, `book2idx`: Creates integer mappings for user/book IDs, essential for embedding layers in neural networks.
    * `user_hist_idx`: Creates a dictionary mapping user *index* to a list of book *indices* they have rated. This interaction history is used as input to the user tower.
* **Dataset & DataLoader:**
    * `TwoTowerDataset`: A PyTorch `Dataset` to load data efficiently. `__getitem__` returns the user index, their history (as a list of book indices), the target book index, and the rating.
    * Standard `train_test_split` on the original `ratings` DataFrame.
    * `DataLoader`: Batches the data. `collate_fn=lambda batch: batch` is used here, meaning padding of histories likely happens inside the model's `forward` method.
* **Model Definition (`TwoTowerModel`):**
    * Inherits from `nn.Module`.
    * `__init__`:
        * `nn.Embedding`: Layers to learn dense vector representations (embeddings) for users and books.
        * `user_mlp`: A Multi-Layer Perceptron (MLP) for the user tower. It takes the concatenation of the user's ID embedding and the *mean* of the embeddings of books in their history.
        * `item_mlp`: An MLP for the item tower, taking the target book's embedding.
    * `forward`: Defines how data flows through the model:
        * Gets user ID embedding.
        * Pads histories within the batch to the same length.
        * Gets embeddings for books in the history.
        * Calculates the *mean* embedding of the history sequence (a simple way to summarize history).
        * Concatenates user ID embedding and mean history embedding.
        * Passes the combined vector through the `user_mlp` -> `u_vec`.
        * Gets the target book embedding and passes it through the `item_mlp` -> `i_vec`.
        * Calculates the dot product of `u_vec` and `i_vec` as the predicted score/rating.
* **Model Instantiation:** Creates the model instance.
* **Training Loop:**
    * Sets up device (GPU/CPU), optimizer (`Adam`), and loss function (`MSELoss` for rating prediction).
    * Standard PyTorch training loop: iterates epochs, gets batches, moves data to device, forward pass, calculates loss, backpropagation, optimizer step. Prints training loss per epoch.
* **Inference:**
    * `model.eval()`: Sets the model to evaluation mode (disables dropout, etc.).
    * **Precompute Item Embeddings:** Calculates and stores the embeddings for *all* books using the trained item tower (`item_embs`). This is efficient for generating recommendations later.
    * **`recommend_user` Function:**
        * Takes a `user_id`.
        * Calculates the user's final embedding (`u_vec`) using the trained user tower (user ID emb + mean history emb).
        * Calculates the dot product between this `u_vec` and *all* precomputed `item_embs`.
        * Uses `torch.topk` to find the items (books) with the highest scores.
        * Maps the resulting indices back to book IDs/titles.
    * **Example Usage:** Calls `recommend_user` for specific users to demonstrate recommendation generation.